In [ ]:
from google.colab import drive
import sys
import torch

drive.mount('/content/drive')

sys.path.insert(1,'/content/drive/My Drive/Mestrado/Codigo/Imports/')

Mounted at /content/drive


The cell below is used for the pre-training of the model using one of the 5 enhancement algorithm: HEF, UM, CLAHE, TCDHE e ATACE.

Select the appropriate dataset folder with the 4 sub-folders: Viral Pneumonia, Normal, Lung Opacity and COVID.

The epoch and learning rate used were 100 and 0.0001, respectively.

In [ ]:
import numpy as np
import cv2
import lightdehazeNet
import lightdehazeTestnet
from pretrain import pretrain
from data_loader import folder_data_loader
from enhancement_algorithms import HEF,UM,CLAHE,TCDHE,xrayjapan

enhancement_method_list = ['hef','um','clahe','tcdhe','atace']

enhancement_method = enhancement_method_list[] # Choose the enhancement method to be used for pre-training

model = 'lightdehazeTestnet'

dataset = "TestCXR"

dataset_folder = #"/content/drive/My Drive/Mestrado/Codigo/Datasets/"+dataset+'/'

file_folder = [dataset_folder+"images/Viral Pneumonia/",
              dataset_folder+"images/Normal/",
              dataset_folder+"images/Lung_Opacity/",
              dataset_folder+"images/COVID/"]

epoch = 100

learning_rate = 0.0001

data_loader = folder_data_loader(class_path_list=file_folder, img_size=(224,224), batch_size=1024, train_ratio = 0.5)

torch.cuda.empty_cache()

clipLimit = 2.0

raio = 8

clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=(raio,raio))

paired_dataset = []

dataset_train = ((data_loader.get_item(index=0, mode='train'))[0]*255).astype(np.uint8)

for i in range(0,dataset_train.shape[0]):
    gray_image = dataset_train[i]
    #enhanced_image = clahe.apply(gray_image)
    if enhancement_method == 'clahe':
      enhanced_image = CLAHE(gray_image,2.0,8)
    if enhancement_method == 'um':
      enhanced_image = UM(gray_image,5,2)
    elif enhancement_method == 'hef':
      enhanced_image = HEF(gray_image,20)
    elif enhancement_method == 'xrayjapan':
      enhanced_image = xrayjapan(gray_image)
    elif enhancement_method == 'tcdhe':
      try:
        enhanced_image = TCDHE(gray_image)
      except:
        enhanced_image = grayimage
        print("Error on the TCDHE!")
    paired_dataset.append([gray_image,enhanced_image])

del dataset_train

paired_dataset = (np.asarray(paired_dataset)/255.0).astype(np.single)
print(paired_dataset.shape)

np.random.shuffle(paired_dataset)

training_size = int(paired_dataset.shape[0]*0.8)

print(training_size)

training_dataset = paired_dataset[0:training_size]

validation_dataset = paired_dataset[training_size:paired_dataset.shape[0]]

del paired_dataset

model = 'lightdehazeTestnet'

import importlib
importlib.reload(lightdehazeTestnet)

if model == "lightdehazeNet":
  ld_net = lightdehazeNet.LightDehaze_Net().cuda()
  folder=dataset_folder+'Pretrained_weights/lightdehazeNet/'

elif model == "lightdehazeTestnet":
  ld_net = lightdehazeTestnet.LightDehaze_Testnet().cuda()
  folder=dataset_folder+'Pretrained_weights/lightdehazeTestnet/'

folder = folder+enhancement_method+'/'

weight = pretrain(ld_net,training_dataset,validation_dataset,epoch,learning_rate,folder)

print("Step Done!")

torch.cuda.empty_cache()

The cell bellow installs the pyiqa module necessary for the running of the algorithm.

In [ ]:
pip install pyiqa

The code below prepares the dataloader object.

In [ ]:
from data_loader import folder_data_loader
import numpy as np
import cv2
import lightdehazeNet
import lightdehazeTestnet
from pretrain import pretrain
import IQAEvolNet
import time

dataset = "TestCXR"

dataset_folder = "/content/drive/My Drive/Mestrado/Codigo/Datasets/"+dataset+'/'

file_folder = [dataset_folder+"images/Viral Pneumonia/",
               dataset_folder+"images/Normal/",
               dataset_folder+"images/Lung_Opacity/",
               dataset_folder+"images/COVID/"]

data_loader = folder_data_loader(class_path_list=file_folder, img_size=(224,224), batch_size=32, train_ratio = 0.8, dataset_size_scaling = 0.1)

The cell bellow runs the evolutionary algorithm.

In [ ]:
model = 'lightdehazeTestnet' #Chosen model

metric = 'complete' # Metric folder chosen, in all cases use complete

test = '11' # Weight set to be used

if model == "lightdehazeNet":
  ld_net = lightdehazeNet.LightDehaze_Net().cuda()
  folder=dataset_folder+'Pretrained_weights/lightdehazeNet/'

elif model == "lightdehazeTestnet":
  ld_net = lightdehazeTestnet.LightDehaze_Testnet().cuda()
  folder=dataset_folder+'Pretrained_weights/lightdehazeTestnet/'

starting_population = []

for enhancement_method in ['um','hef','clahe','xrayjapan','tcdhe']:
  for i in range(0,5):
    starting_population.append(torch.load("/content/drive/My Drive/Mestrado/Codigo/Datasets/"+dataset+"/Pretrained_weights/lightdehazeTestnet/"+enhancement_method+"/Epoch"+str(99-10*i)+'.pth'))

population_size = 25

chance_of_mutation = 0.1

noise_intensity = 1

noise_decay = 0.5

batch_size_per_round = 32

apex_threshold = 1.00001

if test == '1':
  metric_declaration = evolution.MetricDeclaration(niqe=1,entropy=1,ssim=1)
if test == '2':
  metric_declaration = evolution.MetricDeclaration(niqe=1,entropy=1,ssim=2)
if test == '3':
  metric_declaration = evolution.MetricDeclaration(niqe=1,entropy=2,ssim=1)
if test == '4':
  metric_declaration = evolution.MetricDeclaration(niqe=2,entropy=1,ssim=1)
if test == '5':
  metric_declaration = evolution.MetricDeclaration(niqe=1,entropy=4,ssim=1)
if test == '6':
  metric_declaration = evolution.MetricDeclaration(niqe=1,entropy=4,ssim=2)
if test == '7':
  metric_declaration = evolution.MetricDeclaration(niqe=1,entropy=8,ssim=1)
if test == '8':
  metric_declaration = evolution.MetricDeclaration(niqe=1,entropy=16,ssim=1)
if test == '9':
  metric_declaration = evolution.MetricDeclaration(niqe=1,entropy=0,ssim=0)
if test == '10':
  metric_declaration = evolution.MetricDeclaration(niqe=0,entropy=1,ssim=0)
if test == '11':
  metric_declaration = evolution.MetricDeclaration(niqe=1,entropy=1,ssim=0)

evolutionProcess = IQAEvolNet.IQAEvolNet(ld_net, starting_population[0:population_size],chance_of_mutation,
		noise_intensity,noise_decay,data_loader,batch_size_per_round,
		apex_threshold,metric_declaration)

chronometer = []
train_results = []
validation_results = []

for i in range(0,50):
  print('==================================================')
  print("Epoch ",i+1)
  start = time.time()
  evolutionProcess.evolve()
  end = time.time()
  chronometer.append(end-start)
  print("Epochs without improvement: ",evolutionProcess.epochs_without_improvement)
  x_val,y_val = evolutionProcess.validation_score()
  x_train = evolutionProcess.apex_score
  y_train = evolutionProcess.apex_fitness_measure_avg
  print(np.append(x_train,y_train))
  print(np.append(x_val,y_val))
  print('==================================================')
  train_results.append(np.append(x_train,y_train))
  validation_results.append(np.append(x_val,y_val))
  if evolutionProcess.epochs_without_improvement == 3:
    break

for i,apex in enumerate(evolutionProcess.return_apex_timeline()):
  torch.save(apex,"/content/drive/My Drive/Mestrado/Codigo/Enhancement/"+dataset+'/'+metric+'/'+test+'/Generation'+str(i)+'.pth')

torch.save(evolutionProcess.apex,"/content/drive/My Drive/Mestrado/Codigo/Enhancement/"+dataset+'/'+metric+'/'+test+'/apex.pth')

np.save("/content/drive/My Drive/Mestrado/Codigo/Enhancement/"+dataset+'/'+metric+'/'+test+'/chronometer',np.asarray(chronometer))

np.save("/content/drive/My Drive/Mestrado/Codigo/Enhancement/"+dataset+'/'+metric+'/'+test+'/train_results',np.asarray(train_results))

np.save("/content/drive/My Drive/Mestrado/Codigo/Enhancement/"+dataset+'/'+metric+'/'+test+'/validation_results',np.asarray(validation_results))